1.将posts和replys数据集导入R中，命名为r.posts和r.replys，保留所有字段名，所有字段空⽩转化为NA。

In [1]:

library(data.table)
library(stringr)
library(lubridate)
r.posts <- fread("posts.csv", na.strings = "", fill = TRUE)  
r.replys <- fread("replys.csv", na.strings = "", fill = TRUE)


Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




2. 对r.posts进⾏操作

删除所有postnums字段都为0的⾏，查看post_id的数量

In [2]:
r.posts <- r.posts[postnums != 0]
post_id_count <- length(unique(r.posts$post_id))
cat("删除后post_id数量：", post_id_count, "\n")

删除后post_id数量： 253092 


统计字段guba_name为**“螺纹钢吧”**帖⼦的数量，与postnums进⾏对比，哪⼀个更⼤。

In [3]:
lwg_posts <- r.posts[guba_name == "螺纹钢吧"]
lwg_count <- nrow(lwg_posts)
lwg_postnums_sum <- sum(lwg_posts$postnums, na.rm = TRUE)
cat("螺纹钢吧帖子数量：", lwg_count, "\n")
cat("螺纹钢吧postnums总和：", lwg_postnums_sum, "\n")
cat("更大的是：", ifelse(lwg_count > lwg_postnums_sum, "帖子数量", "postnums总和"), "\n")

螺纹钢吧帖子数量： 23209 
螺纹钢吧postnums总和： 575977753 
更大的是： postnums总和 


按create_time进⾏升序排列，找出每个吧中最早创建的帖⼦，将所有这些帖⼦的信息导出⽣成⼀个表,命名为earl.posts。

In [14]:
r.posts[, create_time := ymd_hms(create_time, tz = "UTC")]
earl.posts <- r.posts[, .SD[which.min(create_time)], by = guba_name]
print(earl.posts)
fwrite(earl.posts, "earl.posts.csv")


         guba_name postnums   readnum replynum                post_id
            <char>    <int>     <int>    <int>                 <char>
  1:        玉米吧      479 657191970    13537    news,cjpl,419809422
  2:        焦炭吧     3335      1790        5   news,fdcej,233218236
  3:        豆油吧      127     19874       23   news,fdcey,582100623
  4:        豆粕吧      791      2724       14   news,fdcem,106128577
  5:         PTA吧      616   1076205     5310 news,fczceta,138447392
 ---                                                                 
166: (204004)GC004       55     13178        1  news,204004,105441754
167: (204014)GC014       25      9857        0  news,204014,147906319
168: (204028)GC028       18     15099        1  news,204028,142904600
169: (204182)GC182       13     12395        3   news,204182,97370844
170: (204091)GC091       10     11831        1  news,204091,184875721
             create_time                                  author_url
                  <POSc>          

3. 对r.replys进⾏操作：

按照post_id和reply_time进⾏分组升序排列，导出最晚的⼀条回复，⽣成⼀个表，名为late.replys。

In [13]:
r.replys[, reply_time := ymd_hms(reply_time, tz = "UTC")]
late.replys <- r.replys[, .SD[which.max(reply_time)], by = post_id]
print(late.replys)
fwrite(late.replys, "late.replys.csv")

                   post_id     reply_author
                    <char>           <char>
    1: news,cjpl,697586407 n722613480611282
    2: news,cjpl,697592252     郑郑郑陈陈陈
    3: news,cjpl,701098527       股友Sz8qun
    4: news,cjpl,700761006       和平与平和
    5: news,cjpl,701264895         怀玉米米
   ---                                     
80878:   news,qh,658052945 天下无敌空单巨量
80879:   news,qh,657318728          bac4896
80880:   news,qh,658068391 天下无敌空单巨量
80881:   news,qh,656016422       菜粕人生13
80882:   news,qh,656195806       与于发放处
                                  reply_author_url          reply_time
                                            <char>              <POSc>
    1: http://iguba.eastmoney.com/7226013480611282 2017-09-08 10:13:19
    2: http://iguba.eastmoney.com/3277094349502764 2017-09-08 10:24:12
    3: http://iguba.eastmoney.com/3741044721940080 2017-09-08 14:33:37
    4: http://iguba.eastmoney.com/1016113967816066 2017-09-08 15:05:56
    5: http://iguba.eastmoney.com/2890074720

删除所有reply_content中带有’<>’的部分，例如<imgsrc=""http://gbres.dfcfw.com/face/emot/emot14.png""title=""⽜"">，将此类内容删除。

In [ ]:

r.replys[, reply_content := str_remove_all(reply_content, "<.*?>")] 
print(r.replys)

            reply_author                            reply_author_url
                  <char>                                      <char>
     1:   投资逻辑要理清 http://iguba.eastmoney.com/3937024637376220
     2: 东财网友56315... http://iguba.eastmoney.com/5631513300790064
     3:       股友xFiRtj http://iguba.eastmoney.com/8103094277422700
     4:       内吧潜水员 http://iguba.eastmoney.com/7277064708143926
     5:         贴子零张 http://iguba.eastmoney.com/3144114720877854
    ---                                                             
602501:       与于发放处 http://iguba.eastmoney.com/3860094646980536
602502:       与于发放处 http://iguba.eastmoney.com/3860094646980536
602503:       与于发放处 http://iguba.eastmoney.com/3860094646980536
602504:       与于发放处 http://iguba.eastmoney.com/3860094646980536
602505:       与于发放处 http://iguba.eastmoney.com/3860094646980536
                 reply_time
                     <POSc>
     1: 2017-08-30 17:03:17
     2: 2017-08-30 19:36:50
     3: 2017-08-30 19:52:03
     4: 20

4. 上述操作完成之后，对已经处理好的r.posts和r.replys，以post_id这个字段进⾏合并，⽣成⼀个表，命名为int.data。要求每⼀条r条reply都对应⼀条帖⼦的信息，也就是将帖⼦中的reply全部展开，删除在两表中⽆法匹配的条⽬。

In [20]:
int.data <- merge(r.posts, r.replys, by = "post_id", all = FALSE)
print(int.data)
fwrite(int.data, "int.data.csv")

Key: <post_id>
                      post_id    guba_name postnums readnum replynum
                       <char>       <char>    <int>   <int>    <int>
     1: news,201000,155213690 (201000)R003       26   16775        3
     2: news,201000,155213690 (201000)R003       26   16775        3
     3: news,201000,155213690 (201000)R003       26   16775        3
     4: news,201000,156989517 (201000)R003       26   15208        3
     5: news,201000,156989517 (201000)R003       26   15208        3
    ---                                                             
602501:    news,zxb,715196349     中小板吧      535    3533        3
602502:    news,zxb,715513482     中小板吧      535    3384        3
602503:    news,zxb,715513482     中小板吧      535    3384        3
602504:    news,zxb,715513482     中小板吧      535    3384        3
602505:    news,zxb,723601331     中小板吧      535     877        1
                create_time                                  author_url
                     <POSc>         

5. 对int.data进⾏操作：

提取int.data中create_time的年⽉⽇部分，⽣成⼀个新变量create_date，以"%Y-%m-%d"的Date格式储存；将create_time改为POSXct的标准时间格式："%Y-%m-%d %H:%M:%OS"。

In [22]:

int.data[, create_date := as.Date(create_time, format = "%Y-%m-%d")]
print(int.data)


Key: <post_id>
                      post_id    guba_name postnums readnum replynum
                       <char>       <char>    <int>   <int>    <int>
     1: news,201000,155213690 (201000)R003       26   16775        3
     2: news,201000,155213690 (201000)R003       26   16775        3
     3: news,201000,155213690 (201000)R003       26   16775        3
     4: news,201000,156989517 (201000)R003       26   15208        3
     5: news,201000,156989517 (201000)R003       26   15208        3
    ---                                                             
602501:    news,zxb,715196349     中小板吧      535    3533        3
602502:    news,zxb,715513482     中小板吧      535    3384        3
602503:    news,zxb,715513482     中小板吧      535    3384        3
602504:    news,zxb,715513482     中小板吧      535    3384        3
602505:    news,zxb,723601331     中小板吧      535     877        1
                create_time                                  author_url
                     <POSc>         

提取int.data中create_date的最⼤值和最⼩值，提取所有guba_name，对每⼀个guba_name都⽣成⼀个以⽇为单位的时间序列向量，区间为create_date的最⼤值和最⼩值。并最后输出⼀个表，命名为tim。例如有三个guba_name：“⽟米吧”，“螺纹钢吧”，“财经评论吧”，create_date最⼩值和最⼤值分别为“2015-5-15”和“2015-5-17”，则⽣成了9⾏观察值，每个⽇期都分别对应了每个吧

In [23]:
date_min <- min(int.data$create_date, na.rm = TRUE)
date_max <- max(int.data$create_date, na.rm = TRUE)
date_seq <- seq.Date(date_min, date_max, by = "day")
guba_list <- unique(int.data$guba_name)
tim <- CJ(guba_name = guba_list, create_date = date_seq)
print(tim)

Key: <guba_name, create_date>
           guba_name create_date
              <char>      <Date>
     1: (201000)R003  2010-05-13
     2: (201000)R003  2010-05-14
     3: (201000)R003  2010-05-15
     4: (201000)R003  2010-05-16
     5: (201000)R003  2010-05-17
    ---                         
379466:     黑马股吧  2017-10-28
379467:     黑马股吧  2017-10-29
379468:     黑马股吧  2017-10-30
379469:     黑马股吧  2017-10-31
379470:     黑马股吧  2017-11-01


将tim与int.data以guba_name和create_date进⾏合并，⽣成⼀个新表，命名为int.data.ts，将所有为NA的readnum和replynum观测填充为0。

In [24]:
int.data.ts <- merge(tim, int.data, by = c("guba_name", "create_date"), all.x = TRUE)
int.data.ts[, `:=`(readnum = ifelse(is.na(readnum), 0, readnum),replynum = ifelse(is.na(replynum), 0, replynum))]
print(int.data.ts)

Key: <guba_name, create_date>
           guba_name create_date post_id postnums readnum replynum create_time
              <char>      <Date>  <char>    <int>   <num>    <num>      <POSc>
     1: (201000)R003  2010-05-13    <NA>       NA       0        0        <NA>
     2: (201000)R003  2010-05-14    <NA>       NA       0        0        <NA>
     3: (201000)R003  2010-05-15    <NA>       NA       0        0        <NA>
     4: (201000)R003  2010-05-16    <NA>       NA       0        0        <NA>
     5: (201000)R003  2010-05-17    <NA>       NA       0        0        <NA>
    ---                                                                       
975331:     黑马股吧  2017-10-28    <NA>       NA       0        0        <NA>
975332:     黑马股吧  2017-10-29    <NA>       NA       0        0        <NA>
975333:     黑马股吧  2017-10-30    <NA>       NA       0        0        <NA>
975334:     黑马股吧  2017-10-31    <NA>       NA       0        0        <NA>
975335:     黑马股吧  2017-11-01    <NA>  

6. 统计分析

在r.posts中，提取出replynum最多的⼗条帖⼦和⼗个吧，⽣成两张表为mx.post和mx.ba，表⾥变量分别包括帖⼦id、吧名、帖⼦阅读数和帖⼦回复数，吧名、吧总体阅读数和吧总体回复数

In [25]:
mx.post <- r.posts[order(-replynum)][1:10, .(post_id, guba_name, readnum, replynum)]
mx.ba <- r.posts[, .(total_readnum = sum(readnum, na.rm = TRUE),  total_replynum = sum(replynum, na.rm = TRUE)),  by = guba_name][order(-total_replynum)][1:10]
print(mx.post)
print(mx.ba)

fwrite(mx.post, "mx.post.csv")
fwrite(mx.ba, "mx.ba.csv")

                   post_id  guba_name   readnum replynum
                    <char>     <char>     <int>    <int>
 1:    news,cjpl,419809422     玉米吧 657191970    13537
 2:     news,zxb,614002065   中小板吧    764065     7168
 3: news,fczceta,138447392      PTA吧   1076205     5310
 4:    news,gssz,694173220 股市实战吧    560306     3307
 5:    news,hxnc,653675348 核心内参吧    146617     2108
 6:    news,gssz,562574009 股市实战吧    858569     1651
 7:    news,fdcel,99997951   聚乙烯吧    429510     1420
 8:    news,cjpl,196572402 财经评论吧   2235798     1377
 9:    news,cjpl,689006937 财经评论吧   1541968      963
10:    news,gmxx,558012711 股民学校吧    273178      825
     guba_name total_readnum total_replynum
        <char>         <int>          <int>
 1: 财经评论吧    1272101854         387176
 2:   螺纹钢吧      46297191          79748
 3:   铁矿石吧      22181255          41239
 4: 核心内参吧      56217445          30057
 5: 股指期货吧      16714063          23482
 6:   创业板吧      15109666          20577
 7: 股市实战吧      25245648          

在r.posts中，调⽤任何相关的描述性统计包，对帖⼦和吧做描述性统计，要求⾄少导出replynum和readnum的最⼤值、最⼩值、均值、中位数和⽅差，并导成⼀张表名为stst.int。

In [27]:
stst.int <- r.posts[, .(
  replynum_max = max(replynum, na.rm = TRUE),
  replynum_min = min(replynum, na.rm = TRUE),
  replynum_mean = mean(replynum, na.rm = TRUE),
  replynum_median = median(replynum, na.rm = TRUE),
  replynum_var = var(replynum, na.rm = TRUE),
  readnum_max = max(readnum, na.rm = TRUE),
  readnum_min = min(readnum, na.rm = TRUE),
  readnum_mean = mean(readnum, na.rm = TRUE),
  readnum_median = median(readnum, na.rm = TRUE),
  readnum_var = var(readnum, na.rm = TRUE)
)]
print(stst.int)
fwrite(stst.int, "stst.int.csv")

   replynum_max replynum_min replynum_mean replynum_median replynum_var
          <int>        <int>         <num>           <num>        <num>
1:        13537            0      3.039235               0     1219.928
   readnum_max readnum_min readnum_mean readnum_median readnum_var
         <int>       <int>        <num>          <num>       <num>
1:   657191970           3     10115.53           1137 1.77157e+12
